In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import metrics

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import Birch
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MiniBatchKMeans

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv('data/sports.csv')

In [4]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,text,class,id
0,Looking for a new rain/wind jacket. The @Cross...,golf,1193
1,Feeling pretty good about my choice to bandwag...,hockey,1201
2,PCB have appointed Azhar Ali as\nnational cric...,cricket,1221
3,It’s been awhile now. Does anyone know where t...,tabletennis,896
4,Afternoon #tea is an important part of our liv...,polo,1253


In [5]:
df.shape

(2500, 3)

In [6]:
from sklearn import preprocessing

label_encoding = preprocessing.LabelEncoder()

df['class'] = label_encoding.fit_transform(df['class'].astype(str))

df.head()

,text,class,id
0,Looking for a new rain/wind jacket. The @Cross...,1,1193
1,Feeling pretty good about my choice to bandwag...,2,1201
2,PCB have appointed Azhar Ali as\nnational cric...,0,1221
3,It’s been awhile now. Does anyone know where t...,4,896
4,Afternoon #tea is an important part of our liv...,3,1253


In [7]:
df_labels = df['class']

df_labels.sample(10)

423     1
602     1
1937    4
368     1
951     2
1774    3
483     3
2419    4
68      2
2399    2
Name: class, dtype: int64

In [8]:
def build_model(clustering_model, data, labels):
    
    model = clustering_model(data)

    print('homo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    print(50 * '-')
    
    print('%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          %(metrics.homogeneity_score(labels, model.labels_),
            metrics.completeness_score(labels, model.labels_),
            metrics.v_measure_score(labels, model.labels_),
            metrics.adjusted_rand_score(labels, model.labels_),
            metrics.adjusted_mutual_info_score(labels,  model.labels_),
            metrics.silhouette_score(data, model.labels_)))

In [9]:
def k_means(data, n_clusters=5, max_iter=1000):
    model = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(data)
    
    return model

In [10]:
def agglomerative_fn(data, n_clusters=5):
    model = AgglomerativeClustering(n_clusters = n_clusters).fit(data)
    
    return model

In [11]:
def dbscan_fn(data, eps=0.45, min_samples=4):
    model = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    return model

In [12]:
def mean_shift_fn(data, bandwidth=0.85):
    model = MeanShift(bandwidth=bandwidth).fit(data)
    return model

In [13]:
def birch_fn(data, n_clusters=5):
    model = Birch(n_clusters=n_clusters).fit(data)
    return model

In [14]:
def affinity_propagation_fn(data, damping=0.6, max_iter=1000):
    model = AffinityPropagation(damping=damping, max_iter=max_iter).fit(data)
    return model

In [15]:
def mini_batch_kmeans_fn(data, n_clusters=5, max_iter=1000):
    model = MiniBatchKMeans(n_clusters=n_clusters, max_iter=max_iter, batch_size=20).fit(data)
    return model

### TFIDF

In [16]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english')
df_tfidf=tfidf_vectorizer.fit_transform(df['text']) 

In [17]:
df_tfidf

<2500x10666 sparse matrix of type '<class 'numpy.float64'>'
	with 35853 stored elements in Compressed Sparse Row format>

In [18]:
build_model(k_means, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.719	0.803	0.759	0.593	0.758	0.022


In [19]:
build_model(agglomerative_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.218	0.521	0.308	0.088	0.306	0.020


In [20]:
build_model(dbscan_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.008	0.229	0.016	0.000	0.012	0.004


In [21]:
build_model(mean_shift_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
1.000	0.210	0.347	0.002	0.034	0.049


In [22]:
build_model(birch_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.340	0.517	0.410	0.199	0.409	0.019


In [23]:
build_model(affinity_propagation_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.730	0.216	0.333	0.037	0.272	0.050


In [24]:
build_model(mini_batch_kmeans_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.660	0.743	0.699	0.536	0.698	0.022


### Doc2Vec

In [25]:
from getEmbeddings import getEmbeddings
from d2v_pre import d2v_pre

d2v_pre('data/sports.csv')

x,y = getEmbeddings('data/sports_doc2vec.csv')

In [26]:
def doc2vec_build_model(clustering_model, data, labels):
    
    model = clustering_model(data)

    print('homo\tcompl\tv-meas\tARI\tAMI')
    print(50 * '-')
    
    print('%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          %(metrics.homogeneity_score(labels, model.labels_),
            metrics.completeness_score(labels, model.labels_),
            metrics.v_measure_score(labels, model.labels_),
            metrics.adjusted_rand_score(labels, model.labels_),
            metrics.adjusted_mutual_info_score(labels,  model.labels_)))

In [27]:
doc2vec_build_model(k_means, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.042	0.044	0.043	0.030	0.041


In [28]:
doc2vec_build_model(agglomerative_fn, x,y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.034	0.043	0.038	0.023	0.036


In [29]:
doc2vec_build_model(dbscan_fn, x,y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [30]:
doc2vec_build_model(mean_shift_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [31]:
doc2vec_build_model(birch_fn,x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [32]:
doc2vec_build_model(affinity_propagation_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.103	0.035	0.052	0.001	0.017


In [33]:
doc2vec_build_model(mini_batch_kmeans_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.039	0.041	0.040	0.024	0.038
